In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# Example hyperparameters
hidden_size = 24  # Increase hidden size
learning_rate = 0.5   # Adjust learning rate
num_layers = 2  # Increase number of layers
dropout_rate = 0  # Implement dropout to prevent overfitting
weight_decay = 1e-5

# Define the RNN model with the new hyperparameters
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Updated RPSRNN class to move model to the appropriate device
class RPSRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0):
        super(RPSRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True).to(device)
        self.i2o = nn.Linear(hidden_size, output_size).to(device)
        self.softmax = nn.LogSoftmax(dim=1).to(device)
    
    def forward(self, input, hidden):
        input = input.to(device)
        hidden = hidden.to(device)
        output, hidden = self.rnn(input, hidden)
        output = self.i2o(output[:, -1, :])  # We take the output from the last timestep
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        # Initialize the hidden state with the correct dimensions
        return torch.zeros(self.num_layers, 1, self.hidden_size).to(device)

cuda


In [102]:
# Initialize multiple RNNs with different window sizes
window_sizes = [3, 5, 10, 20, 50, 100]
rnns = {window_size: RPSRNN(input_size=3, hidden_size=hidden_size, output_size=3, num_layers=num_layers, dropout=dropout_rate) for window_size in window_sizes}
optimizers = {window_size: optim.SGD(rnn.parameters(), lr=learning_rate, weight_decay=weight_decay) for window_size, rnn in rnns.items()}
accuracies = {window_size: 0 for window_size in window_sizes}  # Tracking accuracies
correct_predictions = {window_size: 0 for window_size in window_sizes}  # Correct predictions count
total_predictions = {window_size: 0 for window_size in window_sizes}  # Total predictions count

# Other global variables
hidden = {window_size: rnn.initHidden() for window_size, rnn in rnns.items()}
previous_moves = []  # Keep track of the player's previous moves
total_matches = 0
wins = 0
losses = 0
draws = 0

# Create buttons with cyan color
button_rock = widgets.Button(description="Rock", style={'button_color': 'cyan'})
button_paper = widgets.Button(description="Paper", style={'button_color': 'cyan'})
button_scissors = widgets.Button(description="Scissors", style={'button_color': 'cyan'})
output = widgets.Output()

# Mapping of winning moves
winning_move = {
    "rock": "paper",
    "paper": "scissors",
    "scissors": "rock"
}

In [103]:
# Convert moves to tensors
def move_to_tensor(move):
    if move == "rock":
        return torch.tensor([1, 0, 0], dtype=torch.float).view(1, -1)
    elif move == "paper":
        return torch.tensor([0, 1, 0], dtype=torch.float).view(1, -1)
    elif move == "scissors":
        return torch.tensor([0, 0, 1], dtype=torch.float).view(1, -1)

def tensor_to_move(tensor):
    _, index = tensor.topk(1)
    move = ["rock", "paper", "scissors"][index.item()]
    return move

# Game logic to determine the winner
def get_winner(player_move, ai_move):
    if player_move == ai_move:
        return "It's a tie!"
    elif (player_move == "rock" and ai_move == "scissors") or \
         (player_move == "paper" and ai_move == "rock") or \
         (player_move == "scissors" and ai_move == "paper"):
        return "You win!"
    else:
        return "AI wins!"

# Training function for each RNN
def train_rnn(rnn, optimizer, move_sequence, player_move):
    rnn.train()  # Ensure the model is in training mode
    hidden_state = rnn.initHidden()
    target_tensor = torch.tensor([["rock", "paper", "scissors"].index(player_move)], dtype=torch.long).to(device)
    
    for move in move_sequence:
        move_tensor = move_to_tensor(move).view(1, 1, -1).to(device)  # Move input to the correct device
        output, hidden_state = rnn(move_tensor, hidden_state)
    
    optimizer.zero_grad()
    loss = nn.NLLLoss()(output, target_tensor)
    loss.backward()
    
    # Clip gradients to prevent exploding gradients
    torch.nn.utils.clip_grad_norm_(rnn.parameters(), max_norm=1.0)
    
    optimizer.step()

# Update the accuracy of each RNN
def update_accuracies(predicted_move, actual_move, window_size):
    total_predictions[window_size] += 1
    if predicted_move == actual_move:
        correct_predictions[window_size] += 1
    #print(f"Window size: {window_size}, Correct: {correct_predictions[window_size]}, Total: {total_predictions[window_size]}")
    accuracies[window_size] = correct_predictions[window_size] / total_predictions[window_size]

# Predict with each RNN and return their predictions
def predict_with_rnns(previous_moves):
    predictions = {}
    
    for window_size, rnn in rnns.items():
        if len(previous_moves) >= window_size:
            move_sequence = previous_moves[-window_size:]
            hidden_state = rnn.initHidden()
            
            rnn.eval()  # Set model to evaluation mode
            with torch.no_grad():  # No need to compute gradients during inference
                for move in move_sequence:
                    move_tensor = move_to_tensor(move).view(1, 1, -1).to(device)  # Move input to the correct device
                    output, hidden_state = rnn(move_tensor, hidden_state)
                
            predicted_move = tensor_to_move(output)
            predictions[window_size] = predicted_move
    
    return predictions

# Weighted voting based on RNN accuracies
def weighted_vote(predictions):
    vote_counts = {"rock": 0, "paper": 0, "scissors": 0}
    
    for window_size, prediction in predictions.items():
        weight = accuracies[window_size] + 0.1  # Add a small constant to avoid zero weights
        vote_counts[prediction] += weight
    
    return max(vote_counts, key=vote_counts.get)

def process_ai_move(previous_moves, player_move, rnns, optimizers, winning_move):
    if len(previous_moves) > 0:
        predictions = predict_with_rnns(previous_moves)
        
        # Predict the player's next move based on the RNNs
        predicted_player_move = weighted_vote(predictions)  
        ai_move = winning_move[predicted_player_move]  # AI chooses the move that beats the predicted move
        
        # Update RNNs and their accuracies
        for window_size, rnn in rnns.items():
            if len(previous_moves) >= window_size:
                move_sequence = previous_moves[-window_size:]
                train_rnn(rnn, optimizers[window_size], move_sequence, player_move)
                update_accuracies(predictions[window_size], player_move, window_size)
    else:
        ai_move = random.choice(["rock", "paper", "scissors"])
    
    return ai_move


In [104]:
def on_button_click(b):
    global wins; global losses; global draws; global total_matches
    player_move = b.description.lower()
    
    with output:
        clear_output()
        
        ai_move = process_ai_move(previous_moves, player_move, rnns, optimizers, winning_move)
        
        result = get_winner(player_move, ai_move)
        
        if result == "AI wins!":
            losses += 1
        elif result == "You win!":
            wins += 1
        else:
            draws += 1

        total_matches = losses + wins + draws
        win_rate = (wins / total_matches) * 100 if total_matches > 0 else 0
        wlr = wins/losses
        print(f"You chose: {player_move}")
        print(f"AI chose: {ai_move}")
        print(result)
        print(f"\nTotal Matches: {total_matches}")
        print(f"Wins: {wins} | Losses: {losses} | Draws: {draws}")
        print(f"Win Rate: {win_rate:.2f}%")
        print(f"Win/Loss Rate: {wlr:.2f}")
    
    previous_moves.append(player_move)



# Bind buttons to the event handler and display
button_rock.on_click(on_button_click)
button_paper.on_click(on_button_click)
button_scissors.on_click(on_button_click)
display(widgets.HBox([button_rock, button_paper, button_scissors]), output)

Output()

In [105]:
import random
from tqdm import tqdm

# Possible moves
moves = ["rock", "paper", "scissors"]

# Function to generate a random move
def random_move(move_sequence):
    return random.choice(moves)

def silly_strategy(move_sequence):
    if len(move_sequence) == 0:
        return random.choice(["rock", "paper", "scissors"])  # Random move if it's the first round
    
    last_move = move_sequence[-1]
    
    if last_move == "rock":
        return "paper"
    elif last_move == "paper":
        return "scissors"
    elif last_move == "scissors":
        return "rock"
# Function to allow for a customizable strategy
def human_move(move_sequence, strategy=random_move):
    # strategy can be a function that takes move_sequence as input and returns a move
    return strategy(move_sequence)

# Function to simulate N matches between AI and the opponent
def simulate_matches(N, opponent_strategy=silly_strategy):
    previous_moves = []
    
    ai_wins = 0
    opponent_wins = 0
    total_draws = 0
    
    for _ in tqdm(range(N)):
        opponent_move = human_move(previous_moves, strategy=opponent_strategy)
        
        ai_move = process_ai_move(previous_moves, opponent_move, rnns, optimizers, winning_move)
        
        result = get_winner(opponent_move, ai_move)
        
        if result == "AI wins!":
            ai_wins += 1
        elif result == "You win!":
            opponent_wins += 1
        else:
            total_draws += 1
        
        previous_moves.append(opponent_move)
    
    print(f"After {N} matches:")
    print(f"AI Wins: {ai_wins}")
    print(f"Opponent Wins: {opponent_wins}")
    print(f"Draws: {total_draws}")
    print(f"AI Win Rate: {(ai_wins / N) * 100:.2f}%")


# Example usage: simulate 100 matches with a random opponent
#simulate_matches(1000)
